### Cab-Driver Agent

In [ ]:
# Importing libraries
import numpy as np
import random
import math
from collections import deque
import collections
import pickle

# for building DQN model
from keras import layers
from keras import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.optimizers import Adam
# for plotting graphs
import matplotlib.pyplot as plt
import os


In [ ]:
# Import the environment
from Env import CabDriver




In [ ]:
env=CabDriver()

#### Defining Time Matrix

In [ ]:
# Loading the time matrix provided
Time_matrix = np.load("TM.npy")






#### Tracking the state-action pairs for checking convergence


In [ ]:
# Initialise Q_dictionary as 'Q_dict' and States_tracked as 'States_track' (for convergence)
Q_dict = collections.defaultdict(dict)
States_track = collections.defaultdict(dict)



In [ ]:
def initialise_tracking_states():
    sample_q_values = [((2,4,5),(2,1)),((3,23,5),(2,1))]    
    for q_values in sample_q_values:
        state = q_values[0]
        action = q_values[1]
        States_track[state][action] = []
        
        
        
        

In [ ]:
initialise_tracking_states()

In [ ]:
States_track

In [ ]:
#Defining a function to save the Q-dictionary as a pickle file
def save_obj(obj, name ):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

### Agent Class

If you are using this framework, you need to fill the following to complete the following code block:
1. State and Action Size
2. Hyperparameters
3. Create a neural-network model in function 'build_model()'
4. Define epsilon-greedy strategy in function 'get_action()'
5. Complete the function 'append_sample()'. This function appends the recent experience tuple <state, action, reward, new-state> to the memory
6. Complete the 'train_model()' function with following logic:
   - If the memory size is greater than mini-batch size, you randomly sample experiences from memory as per the mini-batch size and do the following:
      - Initialise your input and output batch for training the model
      - Calculate the target Q value for each sample: reward + gamma*max(Q(s'a,))
      - Get Q(s', a) values from the last trained model
      - Update the input batch as your encoded state and output batch as your Q-values
      - Then fit your DQN model using the updated input and output batch.

In [ ]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        # Define size of state and action
        self.state_size = state_size
        self.action_size = action_size

        # Write here: Specify you hyper parameters for the DQN
        self.discount_factor =0.95
        self.learning_rate = 0.01       
        self.epsilon = 1
        self.epsilon_decay = 0.999
        self.epsilon_min = 0.0001
        
        self.batch_size = 5        
        # create replay memory using deque
        self.memory = deque(maxlen=2000)

        # create main model and target model
        self.model = self.build_model()

    # approximate Q function using Neural Network
    def build_model(self):
        model = Sequential()
        # Write your code here: Add layers to your neural nets       
         # hidden layers
        model.add(Dense(32, input_dim=self.state_size, activation='relu', kernel_initializer='he_uniform'))
        model.add(Dense(32, activation='relu', kernel_initializer='he_uniform'))

        # the output layer: output is of size num_actions
        model.add(Dense(self.action_size, activation='relu', kernel_initializer='he_uniform'))

        model.compile(loss='mse',optimizer=Adam(lr=self.learning_rate))
        model.summary
        return model



    def get_action(self, state):
    # Write your code here:
    # get action from model using epsilon-greedy policy
    # Decay in ε after we generate each sample from the environment       
         if np.random.rand() <= self.epsilon:
            # explore: choose a random action from all possible actions
            actions_in_current_state=env.requests(state)
            #print("list of actions.....",actions_in_current_state)
            return random.choice(actions_in_current_state[0])
         else:
            # choose the action with the highest q(s, a)
            # the first index corresponds to the batch size, so
            # reshape state to (1, state_size) so that the first index corresponds to the batch size
            state_epsilon = env.state_encod_arch1(state)
            update_input_eps = np.zeros((1, self.state_size))
            update_input_eps[0]=state_epsilon
            q_value = self.model.predict(update_input_eps)
            #print("eps q value....",q_value,np.argmax(q_value[0]))
            return np.argmax(q_value[0])
        
        
        
        


    def append_sample(self, state, action, reward, next_state,done):
    # Write your code here:
    # save sample <s,a,r,s'> to the replay memory
        self.memory.append((state, action, reward, next_state, done))
    
    
    # pick samples randomly from replay memory (with batch_size) and train the network
    def train_model(self):
        
        if len(self.memory) > self.batch_size:
            # Sample batch from the memory
            mini_batch = random.sample(self.memory, self.batch_size)
            update_output= np.zeros((self.batch_size, self.state_size))
            update_input = np.zeros((self.batch_size, self.state_size))
            #print("output shape....",update_output.shape)
            actions, rewards,done = [], [],[]
            
            for i in range(self.batch_size):
                state, action, reward, next_state,done_boolean = mini_batch[i]
                input_state=env.state_encod_arch1(state)
                #print("input shape....",len(input_state))
                update_input[i] = input_state
                actions.append(action)
                rewards.append(reward)
                output_state=env.state_encod_arch1(next_state)
                update_output[i] = output_state
                done.append(done_boolean)
                
                # Write your code from here
                # 1. Predict the target from earlier model
            target = self.model.predict(update_input)
                    
                # 2. Get the target for the Q-network
            target_qval = self.model.predict(update_output)
                
                #3. Update your 'update_output' and 'update_input' batch
            for i in range(self.batch_size):
                action_value_index=[j for j in range(len(env.action_space)) if env.action_space[j]==actions[i]][0]
                if done[i]:
                    target[i][action_value_index] = rewards[i]
                else: # non-terminal state
                    #print("actions of i:",actions[i])
                    target[i][action_value_index] = rewards[i] + self.discount_factor * np.max(target_qval[i])                
                
                # 4. Fit your model and track the loss values
            self.model.fit(update_input, target, batch_size=self.batch_size, epochs=1, verbose=0)


    def save(self, name):
        self.model.save(name)

In [ ]:
Episodes = 4000

rewards_per_episode=[]
episodes=[]
epsilon_value=1.0
state_size = len(env.state_space)
action_size = len(env.action_space)
agent=DQNAgent(state_size=36,action_size=action_size)



## Epsilon decay function

In [ ]:
time = np.arange(0,10000)
tmpepsilon=1
epsilon = []
for i in range(0,10000):
    tmpepsilon=tmpepsilon*0.999
    epsilon.append(tmpepsilon)
    
    
    

In [ ]:
plt.plot(time, epsilon)
plt.show()


In [ ]:
epsilon[9999]

In [ ]:
np.random.rand()

### DQN block

In [ ]:
for episode in range(Episodes):

    # Write code here
    # Call the environment
    env=CabDriver()
    # Call all the initialised variables of the environment
    state_size = len(env.state_space)
    action_size = len(env.action_space)
    state=env.state_init
    score=0

    
    #Call the DQN agent
    terminal_state=False
    while terminal_state==False:
        
        # Write your code here
        # 1. Pick epsilon-greedy action from possible actions for the current state
        action = env.action_space[agent.get_action(state)]

        # 2. Evaluate your reward and next state
        reward=env.reward_func(state,action,Time_matrix)
        next_state,done= env.next_state_func(state,action,Time_matrix)
        
        # 3. Append the experience to the memory
        agent.append_sample(state, action, reward, next_state, done)
        
        
        # 4. Train the model by calling function agent.train_model
        agent.train_model()
        
        
        # 5. Keep a track of rewards, Q-values, loss
        score += reward
        state = next_state
        terminal_state=done

    rewards_per_episode.append(score)
    episodes.append(episode)
    
    
    # epsilon decay
    if agent.epsilon > agent.epsilon_min:
        agent.epsilon *= agent.epsilon_decay
        #print("epsilon value....",agent.epsilon)

    # every episode:
    print("episode {0}, reward {1}, memory_length {2}, epsilon {3}".format(episode,
                                                                         score,
                                                                         len(agent.memory),
                                                                         agent.epsilon))
    # every few episodes:
    #if episode % 100 == 0:
        # store q-values of some prespecified state-action pairs
        # q_dict = agent.store_q_values()

        # save model weights
        #agent.save(name="model_weights.h5")
    

# save stuff as pickle
def save_pickle(obj, name):
    with open(name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

# make directory
if not os.path.exists("saved_pickle_files"):
    os.mkdir("saved_pickle_files")

# save rewards_per_episode
save_pickle(rewards_per_episode, "saved_pickle_files/rewards_per_episode")


# plot results
#with open('saved_pickle_files/rewards_per_episode.pkl', 'rb') as f:
#    rewards_per_episode = pickle.load(f)

plt.plot(list(range(len(rewards_per_episode))), rewards_per_episode)
plt.xlabel("episode number")
plt.ylabel("reward per episode")

# save plots in saved_plots/ directory
plt.savefig('rewards.png')

print("Average reward of last 100 episodes is {0}".format(np.mean(rewards_per_episode[-100:]))) 


In [ ]:
[i for i in range(len(env.action_space)) if env.action_space[i]==(1,0)][0]

In [ ]:
env.action_space

### Tracking Convergence

#### Epsilon-decay sample function

<div class="alert alert-block alert-info">
Try building a similar epsilon-decay function for your model.
</div>

In [ ]:
time = np.arange(0,10000)
epsilon = []
for i in range(0,10000):
    epsilon.append(0 + (1 - 0) * np.exp(-0.0009*i))
    
    
    
    

In [ ]:
plt.plot(time, epsilon)
plt.show()




In [ ]:
from gym import spaces

In [ ]:
spaces.Discrete(15+1)

In [ ]:
import numpy as np

In [ ]:
np.random.choice(np.arange(0,15+1)) 

In [ ]:
m = 5 # number of cities, ranges from 1 ..... m
t = 24 # number of hours, ranges from 0 .... t-1
d = 7  # number of days, ranges from 0 ... d-1
C = 5 # Per hour fuel and other costs
R = 9 # per hour revenue from a passenger

In [ ]:
np.arange(0,15+1)

In [ ]:
from itertools import product

In [ ]:
listi=list(product(np.arange(1,m+1),np.arange(0,t),np.arange(0,d)))

In [ ]:
len(listi)

In [ ]:
state_tmp=listi[np.random.choice(np.arange(0,len(listi)+1))]

In [ ]:
listi_init

In [ ]:
st=[0]*m

In [ ]:
st[listi_init[0]-1]=1

In [ ]:
st+st

In [ ]:
 [tup for tup in list(product(np.arange(1,m+1),np.arange(1,m+1))) if tup[0]!=tup[1]]

In [ ]:
npyfile=np.load("TM.npy")

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
npyfile[1][4]